In [ ]:
import os
import shapely
import fiona
import geopandas as gpd
import pandas as pd
import matplotlib as plt
from pysld.style import StyleSld
from geo.Geoserver import Geoserver

In [ ]:
!rm -r /home/ajai-krishna/work/qgis_geoserver/Styles/State_styles/*

In [ ]:
data_dir='/home/ajai-krishna/work/qgis_geoserver/India states data'
data_dir_dist = '/home/ajai-krishna/work/qgis_geoserver/India district Data'
files_path=[os.path.join(data_dir,i) for i in os.listdir(data_dir)]
save_state_dir=os.path.join('/home/ajai-krishna/work/qgis_geoserver/Styles/State_styles/')
style_state_dir='/home/ajai-krishna/work/qgis_geoserver/Styles/State_styles'
style_files = [os.path.join(style_state_dir,i) for i in os.listdir(style_state_dir)]


def prepare_qgis_paths(states_dir, districts_dir, styles_dir):
    files_path = [os.path.join(states_dir, f) for f in os.listdir(states_dir)]
    files_path_dist = [os.path.join(districts_dir, f) for f in os.listdir(districts_dir)]
    style_files = [os.path.join(styles_dir, f) for f in os.listdir(styles_dir)]

    return {
        "state_files": files_path,
        "district_files": files_path_dist,
        "styles": style_files,
        "save_dir": styles_dir
    }


In [ ]:
len(files_path)

paths = prepare_qgis_paths(
    '/home/ajai-krishna/work/qgis_geoserver/India states data',
    '/home/ajai-krishna/work/qgis_geoserver/India district Data',
    '/home/ajai-krishna/work/qgis_geoserver/Styles'
)

state_files = paths["state_files"]
# district_files = paths["district_files"]
style_files = paths["styles"]
save_dir = paths["save_dir"]

gdfs = [gpd.read_file(file_path) for file_path in files_path]
data = pd.concat(gdfs, ignore_index=True)


In [ ]:
server = Geoserver('http://localhost:8080/geoserver', username='admin', password='geoserver')

In [ ]:
for i in files_path:
    pr_indices = ['jjas','ond','pr-mm','rainyday-frequency','simple-daily-indices']
    file_name = '_'.join(i.split('_')[-4:-1])
    print(file_name)

In [ ]:

stores={}
styles={}

for i in files_path:
    pr_indices = ['jjas','ond','pr-mm','rainyday-frequency','simple-daily-indices']
    file_name = '_'.join(i.split('_')[-4:-1])
    
    datastore_name = os.path.splitext(os.path.basename(i))[0]
    try:
        server.delete_featurestore(featurestore_name = datastore_name, workspace = 'Climate_data_Indian_States')
        print(f'deleted {datastore_name}')
    except:
        print(f"good")
    store = server.create_gpkg_datastore(path = i)
    stores[datastore_name] = store
    print(f"created data_store:{datastore_name}")

    gdf = gpd.read_file(i)
    layers = fiona.listlayers(i)
    layer_name = layers[0] if layers else datastore_name
    
    # Decide palette
    if file_name.split("_")[1].lower().startswith(tuple(pr_indices)):
        palette = 'Greens'
    else:
        palette = 'Reds'

    sld = StyleSld(
        style_name='polygonStyle',
        geom_type='polygon',
        attribute_name='mean',
        values=gdf['mean'].to_list(),
        number_of_class= 5,
        classification_method='equal_interval',
        color_palette=palette
    )

    style = sld.generate_categorized_style()
    # style=sld.generate_classified_style()
    

    sld_path = os.path.join(save_state_dir, f"simple_style_{file_name}.sld")
    with open(sld_path, 'w') as f:
        f.write(style)
    print(f"✅ Created SLD for {file_name} → {palette}")
    
    try:
        server.delete_style(style_name = file_name, workspace='Climate_data_Indian_States')
        print(f'deleted {file_name}')
    except Exception:
        print(f'style {file_name} not found, skipping delete')   
    
    style1 = server.upload_style(path = sld_path, name = file_name, workspace = 'Climate_data_Indian_States')
    styles[file_name] = style1
    print(f'style uploaded {file_name}')
    
    # layer_name = datastore_name
    server.publish_style(layer_name= layer_name, style_name = file_name ,workspace = 'Climate_data_Indian_States',)
    print(f'published {layer_name} to {datastore_name}')
